PPT Generation Using NLP
Utilize Python with libraries like NLTK or spaCy for keyword extraction and text processing to transform user input into slide content, and then leverage the python-pptx library to dynamically create PowerPoint slides by adding titles, content, and optional images or graphs based on the processed keywords.

In [4]:
pip install SpeechRecognition

In [2]:
pip install pyaudio

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 220.2 kB/s eta 0:00:59
     --------------------------------------- 0.1/12.8 MB 328.2 kB/s eta 0:00:39
     --------------------------------------- 0.1/12.8 MB 419.2 kB/s eta 0:00:31
      -------------------------------------- 0.2/12.8 MB 737.3 kB/s eta 0:00:18
     - -------------------------------------- 0.3/12.8 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.5/12.8 MB 1.4 MB/s eta 0:00:09
     -- ------------------------------------- 0.7/12.8 MB 1.7 MB/s eta 0:00:08
     -- ------------------------------------- 0.8/12.8 MB 1.8 MB/s eta 0:00:07
     -- ------------------------------------- 0.9/12.8 MB 1.9 MB/s eta 0:00:

In [4]:
!pip install python-pptx

In [5]:
!pip install SpeechRecognition pyaudio python-pptx

In [6]:
!pip install wikipedia-api

In [5]:
import speech_recognition as sr
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
import requests

# Initialize a new presentation
prs = Presentation()
commands_file = "commands.txt"  # File to store commands

# Function to recognize speech and convert it to text
def recognize_speech():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for voice command...")
        audio = r.listen(source)
        try:
            command = r.recognize_google(audio)
            print(f"Recognized: {command}")
            # Store the command in a file
            with open(commands_file, "a") as f:
                f.write(command + "\n")
            return command.lower()
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
            return ""
        except sr.RequestError:
            print("Request to Google API failed.")
            return ""

# Function to limit content to approximately 100 words
def limit_content(content, word_limit=100):
    words = content.split()
    limited_content = ' '.join(words[:word_limit])
    return limited_content

# Function to create a slide with a title and content
def create_slide(topic, content=""):
    slide_layout = prs.slide_layouts[1]  # Title and content layout
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = f"Slide about {topic}"
    
    # Limit content to 100 words
    limited_content = limit_content(content)
    
    # Add content to the slide
    content_box = slide.shapes.placeholders[1]
    content_box.text = limited_content
    
    # Set font size for the content
    for paragraph in content_box.text_frame.paragraphs:
        for run in paragraph.runs:
            run.font.size = Pt(20)
            run.font.color.rgb = RGBColor(0, 0, 0)

    print(f"Created a slide about {topic} with limited content.")

# Function to fetch content and image from Wikipedia using the API
def fetch_wikipedia_content_and_image(topic):
    api_url = "https://en.wikipedia.org/w/api.php"
    
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts|pageimages",
        "exintro": True,
        "explaintext": True,
        "titles": topic,
        "redirects": 1
    }

    response = requests.get(api_url, params=params)
    
    if response.status_code != 200:
        print("Error fetching data from Wikipedia.")
        return "Content not found.", None

    data = response.json()
    pages = data.get("query", {}).get("pages", {})
    page_info = next(iter(pages.values()))

    content = page_info.get("extract", "Content not found.")
    image_url = page_info.get("thumbnail", {}).get("source")

    if not image_url:
        print(f"No image found for {topic}.")
    else:
        print(f"Image found: {image_url}")

    return content, image_url

# Function to insert an image into the slide
def insert_image(image_url):
    slide_layout = prs.slide_layouts[1]  # Title and content layout
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = "Image Slide"

    # Download and insert the image
    try:
        image_response = requests.get(image_url)
        image_response.raise_for_status()
        with open("temp_image.jpg", "wb") as f:
            f.write(image_response.content)
        slide.shapes.add_picture("temp_image.jpg", Inches(1), Inches(2), Inches(5), Inches(3))
        print(f"Inserted image from: {image_url}")
    except requests.HTTPError:
        print(f"Failed to download image from: {image_url}")
    except Exception as e:
        print(f"An error occurred while downloading the image: {e}")

# Function to add a text box to the last slide
def add_text_box(text):
    slide = prs.slides[-1]  # Get the last slide
    left = Inches(1)
    top = Inches(1)
    width = Inches(8)
    height = Inches(3)

    textbox = slide.shapes.add_textbox(left, top, width, height)
    text_frame = textbox.text_frame
    p = text_frame.add_paragraph()
    p.text = text
    p.font.size = Pt(20)
    p.font.color.rgb = RGBColor(0, 0, 0)

    print(f"Added a text box with the text: {text}")

# Function to add a table to the last slide
def add_table(rows, cols):
    slide = prs.slides[-1]  # Get the last slide
    left = Inches(1)
    top = Inches(1.5)
    width = Inches(8)
    height = Inches(3)

    table = slide.shapes.add_table(rows, cols, left, top, width, height).table
    for row in range(rows):
        for col in range(cols):
            cell = table.cell(row, col)
            cell.text = f"Row {row + 1}, Col {col + 1}"
            cell.text_frame.paragraphs[0].font.size = Pt(20)
            cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)

    print(f"Added a table with {rows} rows and {cols} columns.")


# Function to add a chart to the slide
def add_chart():
    slide_layout = prs.slide_layouts[5]  # Use a blank layout for charts
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = "Chart Slide"

    # Placeholder for chart creation
    # Inserting text as placeholder for chart
    left = Inches(1)
    top = Inches(2)
    width = Inches(5)
    height = Inches(3)
    textbox = slide.shapes.add_textbox(left, top, width, height)
    text_frame = textbox.text_frame
    p = text_frame.add_paragraph()
    p.text = "Chart Placeholder (Actual chart requires specific data)"
    p.font.size = Pt(20)
    p.font.color.rgb = RGBColor(0, 0, 0)

    print("Added a chart placeholder to the slide.")

# Function to process the latest command
def process_latest_command(command):
    command = command.strip().lower()
    print(f"Processing command: {command}")

    if "exit" in command or "stop" in command:
        print("Exiting the program.")
        return "exit"  # Return a specific value to indicate exit
    elif "create slide" in command:
        topic = command.replace("create slide of", "").strip()
        content, image_url = fetch_wikipedia_content_and_image(topic)
        create_slide(topic, content)
        if image_url:
            insert_image(image_url)
    elif "add text box" in command:
        text = command.replace("add text box:", "").strip()
        add_text_box(text)
    elif "add table" in command:
        add_table(2, 2)  # Example: Add a 2x2 table
    elif "add chart" in command:
        add_chart()
    else:
        print(f"Unrecognized command: {command}")
# Main loop to continuously listen for voice commands
def main():
    print("Starting voice-activated PowerPoint assistant...")
    while True:
        command = recognize_speech()
        if command:
            result = process_latest_command(command)
            if result == "exit":
                break

    # Save the PowerPoint presentation after the session ends
    prs.save("voice_presentation.pptx")
    print("PowerPoint presentation saved as 'voice_presentation.pptx'.")

if __name__ == "__main__":
    main()


Starting voice-activated PowerPoint assistant...
Listening for voice command...
Recognized: create slide of elephant
Processing command: create slide of elephant
Image found: https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/African_Bush_Elephant.jpg/50px-African_Bush_Elephant.jpg
Created a slide about elephant with limited content.
Inserted image from: https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/African_Bush_Elephant.jpg/50px-African_Bush_Elephant.jpg
Listening for voice command...
Recognized: create slide of Apple
Processing command: create slide of apple
Image found: https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/Pink_lady_and_cross_section.jpg/50px-Pink_lady_and_cross_section.jpg
Created a slide about apple with limited content.
Failed to download image from: https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/Pink_lady_and_cross_section.jpg/50px-Pink_lady_and_cross_section.jpg
Listening for voice command...
Recognized: create slide of elephan

In [6]:
import os

# Open the PowerPoint file after creation (for Windows users)
os.startfile('voice_presentation.pptx')

In [9]:
pip install scikit-learn joblib


In [12]:
pip install numpy<2


Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.


In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import joblib

# Load a sample dataset (Iris dataset)
data = load_iris()
X = data.data
y = data.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Save the trained model to a file
model_filename = 'trained_model.joblib'
joblib.dump(model, model_filename)

print(f"Model saved to {model_filename}.")


Model saved to trained_model.joblib.


In [3]:
# Load the model from the file
loaded_model = joblib.load('trained_model.joblib')

# Test the loaded model
test_score = loaded_model.score(X_test, y_test)
print(f"Loaded model accuracy: {test_score:.2f}")


Loaded model accuracy: 1.00
